In [104]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.preprocessing import LabelEncoder


In [108]:
# Import
df_10_consumption = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_010_consumption.csv")
df_10_holiday = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_010_holiday.csv")
df_10_weather = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_010_weather.csv")
df_30_consumption = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_030_consumption.csv")
df_30_holiday = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_030_holiday.csv")
df_30_weather = pd.read_csv(r"C:\Users\NicholasOliver\OneDrive - amendllc.com\DS_Case_Study\2024-04-ds_cast_study_data\division_030_weather.csv")

In [109]:
# Clean & Prep
df_10_weather['AvgTemp'] = df_10_weather['ProjectedHighTemp'] - ((df_10_weather['ProjectedHighTemp'] - df_10_weather['ProjectedLowTemp'])/2)
df_30_weather['AvgTemp'] = df_30_weather['ProjectedHighTemp'] - ((df_30_weather['ProjectedHighTemp'] - df_30_weather['ProjectedLowTemp'])/2)
df_30_holiday = df_30_holiday.drop(['PeakMonth', 'Event01', 'Event02', 'Event03', 'Event04', 'Event05'], axis=1)
df_10_holiday = df_10_holiday.drop(['PeakMonth', 'Event01', 'Event02', 'Event03', 'Event04', 'Event05'], axis=1)
columns_to_check = ['Christmas', 'July4th', 'SuperBowl', 'Thanksgiving', 'Easter', 'LaborDay', 'MemorialDay', 'NewYearsEve', 'NewYearsDay']
df_30_holiday['Holiday?'] = df_30_holiday[columns_to_check].any(axis=1)
df_10_holiday['Holiday?'] = df_10_holiday[columns_to_check].any(axis=1)
df_10_consumption['TotalConsumption'] = df_10_consumption['TotalConsumption'].astype(int)
df_10_consumption['TotalConsumption'] = df_10_consumption['TotalConsumption'].apply(lambda x:abs(x))
df_10_consumption.drop(df_10_consumption[df_10_consumption['TotalConsumption']==0].index, inplace=True)
df_10_consumption = df_10_consumption.drop('Unnamed: 4', axis=1)
df_30_consumption = df_30_consumption.drop('Unnamed: 4', axis=1)
df_10_weather = df_10_weather.drop(['ProjectedHighTemp','ProjectedLowTemp'],axis=1)
df_30_weather = df_30_weather.drop(['ProjectedHighTemp','ProjectedLowTemp'],axis=1)
df_10_consumption['DayOfTheYear'] = pd.to_datetime(df_10_consumption['DayOfTheYear'])
df_30_consumption['DayOfTheYear'] = pd.to_datetime(df_30_consumption['DayOfTheYear'])
df_10_weather['DayOfTheYear'] = pd.to_datetime(df_10_weather['DayOfTheYear'])
df_30_weather['DayOfTheYear'] = pd.to_datetime(df_30_weather['DayOfTheYear'])
df_30_holiday['DayOfTheYear'] = pd.to_datetime(df_30_holiday['DayOfTheYear'])
df_10_holiday['DayOfTheYear'] = pd.to_datetime(df_10_holiday['DayOfTheYear'])
df_10_consumption['DayOfTheWeek'] = df_10_consumption['DayOfTheYear'].dt.day_name()
df_30_consumption['DayOfTheWeek'] = df_30_consumption['DayOfTheYear'].dt.day_name()

df_10 = pd.merge(pd.merge(df_10_consumption, df_10_weather, how='left',on='DayOfTheYear'), df_10_holiday, on='DayOfTheYear', how='left')
df_30 = pd.merge(pd.merge(df_30_consumption, df_30_weather, how='left',on='DayOfTheYear'), df_30_holiday, on='DayOfTheYear', how='left')
df_10 = df_10.drop('Division_y',axis=1).rename(columns={'Division_x':'Division'})
df_30 = df_30.drop('Division_y',axis=1).rename(columns={'Division_x':'Division'})
df = pd.concat([df_10,df_30], ignore_index=True)

In [ ]:
# Data Exploration


df[df['Holiday?']==False].groupby(['DayOfTheWeek'])['TotalConsumption'].mean().sort_values()
#Thursday, Friday, then Wednesday are biggest days for consumption. There is a > 100 mean difference between all 3 of these means, and the rest of the days.We will group Wed Thu Fri as "Busy Days".
#Median has same order. If you were wondering.



#Testing if there is significant difference between days of the week in predicting total consumption
f_stat, p_value = f_oneway(df[df['DayOfTheWeek'] == 'Monday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Tuesday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Wednesday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Thursday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Friday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Saturday']['TotalConsumption'],
                           df[df['DayOfTheWeek'] == 'Sunday']['TotalConsumption'])

print(f_stat)
print(p_value)
if p_value < 0.05:
    print("There is a significant difference between the groups.")
else:
    print("There is no significant difference between the groups.")
tukey_results = pairwise_tukeyhsd(df['TotalConsumption'], df['DayOfTheWeek'])
print(tukey_results)
#Conclusion: There is significant difference between all days except (Mon, Sat, Sun)
#Our two groups: (Mon, Sat, Sun), (Tuesday), (Wednesday), (Thursday), (Friday). This would be impractical having 5 groups though. We might as well just treat them all differently at that point.

In [143]:
# Clean for linear regression model
def busyday(x):
    if x == 'Wednesday':
        return 1
    if x == 'Thursday':
        return 1
    if x == 'Friday':
        return 1
    else:
        return 0
    

# BusyDay = ['Friday', 'Wednesday', 'Thursday']
df['BusyDay?'] = df['DayOfTheWeek'].apply(busyday)

# Set Holidays to int
df['Christmas'] = df['Christmas'].astype(int)
df['July4th'] = df['July4th'].astype(int)
df['SuperBowl'] = df['SuperBowl'].astype(int)
df['Thanksgiving'] = df['Thanksgiving'].astype(int)
df['Easter'] = df['Easter'].astype(int)
df['LaborDay'] = df['LaborDay'].astype(int)
df['MemorialDay'] = df['MemorialDay'].astype(int)
df['NewYearsEve'] = df['NewYearsEve'].astype(int)
df['NewYearsDay'] = df['NewYearsDay'].astype(int)
df['Holiday?'] = df['Holiday?'].astype(int)
#Should we make the days proceeding the holidays also of value 1? How many days before? Do some more statistical exploration

In [ ]:
#Model 1
Y = df['TotalConsumption']
X = df[['AvgTemp', 'PrecipitationChance']]
X = sm.add_constant(X)
modelv1 = sm.OLS(Y, X).fit()
print(modelv1.summary())

In [ ]:
#Model 2
Y = df['TotalConsumption']
X = df[['AvgTemp', 'PrecipitationChance', 'DayOfTheWeek_Encoded']]
X = sm.add_constant(X)
modelv2 = sm.OLS(Y, X).fit()
print(modelv2.summary())

In [ ]:
#Model 3
Y = df['TotalConsumption']
X = df[['AvgTemp', 'PrecipitationChance', 'BusyDay?', 'Christmas', 'July4th', 'SuperBowl', 'Thanksgiving', 'Easter', 'LaborDay', 'MemorialDay', 'NewYearsEve', 'NewYearsDay']]
X = sm.add_constant(X)
modelv3 = sm.OLS(Y, X).fit()
print(modelv3.summary())

In [ ]:
#Model 4
Y = df['TotalConsumption']
X = df[['AvgTemp', 'PrecipitationChance', 'BusyDay?', 'Holiday?']]
X = sm.add_constant(X)
modelv4 = sm.OLS(Y, X).fit()
print(modelv4.summary())